In [ ]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from load_pvoc_data import load_data, TRAIN_LENGTH

In [ ]:
BATCH_SIZE = 32
EPOCHS = 20
VALIDATION_SPLIT = 0.3

In [ ]:
def preprocessing(img, lbl):
    crop_img = tf.image.central_crop(img, 1)
    resized = tf.image.resize_images(img, (256, 256))
    norm_img = tf.image.per_image_standardization(resized)
    
    one_hot = tf.one_hot(lbl, 20)
    summed = tf.reduce_sum(one_hot, axis=-2)
    multi_hot = tf.where(
        tf.equal(summed, 0), tf.zeros_like(summed, dtype=tf.float32), tf.ones_like(summed, dtype=tf.float32)
    )
    return norm_img, multi_hot

In [ ]:
def train_input_fn():
    train_dataset = tf.data.Dataset.from_generator(
        lambda:load_data("train"),
        (tf.uint8, tf.int32),
        (tf.TensorShape([None, None, 3]), tf.TensorShape([None]))
    ).map(preprocessing).shuffle(10000)
    train_dataset = train_dataset.apply(tf.contrib.data.assert_element_shape((
        [256, 256, 3],
        [20]
    )))
    
    val_length = int(VALIDATION_SPLIT * TRAIN_LENGTH)
    val_dataset = train_dataset.take(val_length).apply(
        tf.contrib.data.batch_and_drop_remainder(BATCH_SIZE))
    train_dataset = train_dataset.skip(val_length).apply(
        tf.contrib.data.batch_and_drop_remainder(BATCH_SIZE)).repeat()

    return train_dataset, val_dataset

In [ ]:
def test_input_fn():
    test_dataset = tf.data.Dataset.from_generator(
        lambda:load_data("test"),
        (tf.uint8, tf.int32),
        (tf.TensorShape([None, None, 3]), tf.TensorShape([None]))
    )
    test_dataset = test_dataset.map(preprocessing).apply(tf.contrib.data.assert_element_shape((
        [256, 256, 3],
        [20]
    )))
    return test_dataset.apply(tf.contrib.data.batch_and_drop_remainder(BATCH_SIZE))

In [ ]:
def conv_layer(inputs, eta, scope_name, filters=32, kernel_size=3, strides=1, plastic=True):
    with tf.variable_scope(scope_name):
        w = tf.get_variable('conv_w', (kernel_size, kernel_size, int(inputs.shape[-1]), filters))
        b = tf.get_variable('conv_b', (filters,))
        if plastic:
            alpha = tf.get_variable('conv_alpha', (kernel_size, kernel_size, int(inputs.shape[-1]), filters))
            hebb = tf.get_variable('conv_hebb', (kernel_size, kernel_size, int(inputs.shape[-1]), filters),
                                   trainable=False, initializer=tf.zeros_initializer)
            w = w + tf.multiply(alpha, hebb)
    
    x = tf.nn.conv2d(input=inputs, filter=w, strides=[1, strides, strides, 1], padding='SAME') + b
    
    if plastic:
        # y is to be the output reshaped so as to be used as a kernel for convolution on input to get Hebbian update
        y = tf.image.resize_images(x, [int(inputs.shape[1])] * 2)
        y = tf.transpose(y, [1, 2, 0, 3])

        # in_mod is the input padded a/c to prev. convolution
        in_mod = tf.pad(inputs, [
            [0, 0],
            *([[int(np.floor((kernel_size - 1) / 2)), int(np.ceil((kernel_size - 1) / 2))]] * 2),
            [0, 0]
        ])
        # in_mod is now modded so as to preserve channels and sum over mini-batch samples for Hebbian update convolution
        in_mod = tf.transpose(in_mod, [3, 1, 2, 0])

        hebb_update = tf.nn.conv2d(input=in_mod, filter=y, strides=([1] * 4), padding='VALID')
        hebb = eta * tf.transpose(hebb_update, [1, 2, 0, 3]) + (1 - eta) * hebb
    
    return tf.nn.relu(tf.layers.batch_normalization(x))

In [ ]:
def residual_block(inputs, filters, scope_name, eta):
    with tf.variable_scope(scope_name):
        x = conv_layer(inputs, eta, scope_name='blk_layer_1', filters=filters, kernel_size=1)
        x = conv_layer(inputs, eta, scope_name='blk_layer_2', filters=(filters * 2))
    return x

In [ ]:
def darknet_block(inputs, filters, repetitions, scope_name, eta):
    with tf.variable_scope(scope_name):
        x = conv_layer(inputs, eta, scope_name='blk_layer_0', filters=filters, strides=2)
        for i in range(repetitions):
            x = residual_block(x, filters / 2, scope_name='blk_rep_' + str(i), eta=eta)
    return x

In [ ]:
def darknet_model(features, labels, mode):  
    features = tf.cast(features, dtype=tf.float32)
    
    eta = tf.get_variable('eta', (1,), initializer=tf.truncated_normal_initializer)
    
    x = conv_layer(features, eta, scope_name='first', filters=32)
    x = darknet_block(x, filters=64, repetitions=1, scope_name='dark_blk_0', eta=eta)
    x = darknet_block(x, filters=128, repetitions=2, scope_name='dark_blk_1', eta=eta)
    x = darknet_block(x, filters=256, repetitions=8, scope_name='dark_blk_2', eta=eta)
    x = darknet_block(x, filters=512, repetitions=8, scope_name='dark_blk_3', eta=eta)
    x = darknet_block(x, filters=1024, repetitions=4, scope_name='dark_blk_4', eta=eta)
    
    x = tf.layers.average_pooling2d(x, pool_size=8, strides=8)
    x = tf.layers.flatten(x)
    x = tf.layers.dense(inputs=x, units=20, kernel_initializer=tf.truncated_normal_initializer())
    
    classes = tf.where(tf.sigmoid(x) >= 0.5, tf.ones_like(x, dtype=tf.float32), tf.zeros_like(x, dtype=tf.float32))
    correct_prediction = tf.equal(classes, labels)
    acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    acc = tf.identity(acc, name='accuracy_tensor')
    
    predictions = {'classes': classes, 'accuracy': acc}
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=x)
    loss = tf.reduce_mean(loss)
    
    tf.summary.scalar('accuracy', acc)
    tf.summary.scalar('loss', loss)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.contrib.estimator.TowerOptimizer(tf.train.AdamOptimizer(1e-4))
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {'accuracy': tf.metrics.accuracy(labels=labels, predictions=classes)}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss,
                                      eval_metric_ops=eval_metric_ops)


In [ ]:
tensors_to_log = {'accuracy': 'accuracy_tensor'}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=10)

In [ ]:
model = tf.estimator.Estimator(
    model_fn=tf.contrib.estimator.replicate_model_fn(darknet_model),
    model_dir='/tmp/tmpfull', config=tf.estimator.RunConfig(
        save_checkpoints_steps=150, save_summary_steps=10, log_step_count_steps=10
    )
)

In [ ]:
validation_hook = tf.contrib.learn.monitors.replace_monitors_with_hooks(
    [tf.contrib.learn.monitors.ValidationMonitor(
        input_fn=lambda:train_input_fn()[1], every_n_steps=100, early_stopping_rounds=10
    )],
    model
)[0]

In [ ]:
max_steps = int(((1 - VALIDATION_SPLIT) * TRAIN_LENGTH / BATCH_SIZE) * EPOCHS)
model.train(input_fn=lambda:train_input_fn()[0], hooks=[logging_hook, validation_hook],
            max_steps=max_steps)

In [ ]:
print(model.evaluate(input_fn=test_input_fn))